In [12]:
import polars as pl

import polars_ts as pts  # noqa

# Create sample dataframe with columns `unique_id`, `ds`, and `y`.
df = (
    pl.scan_parquet("https://datasets-nixtla.s3.amazonaws.com/m4-hourly.parquet")
    #.filter(pl.col("unique_id").is_in(["H1", "H2", "H3"]))
    # .group_by("unique_id").agg(pl.col("y").head(5))
    # .explode("y")
    .head(100000)
    .with_columns((pl.col("y") - pl.mean("y")) / pl.std("y").over("unique_id"))
    .collect()
)
df

unique_id,ds,y
str,i64,f64
"""H1""",1,-42.689703
"""H1""",2,-42.811002
"""H1""",3,-42.811002
"""H1""",4,-42.983374
"""H1""",5,-43.289812
…,…,…
"""H207""",616,-1697.332073
"""H207""",617,-1697.308714
"""H207""",618,-1697.402152


In [13]:
pts.compute_pairwise_dtw(df, df)

id_1,id_2,dtw
str,str,f32
"""H153""","""H161""",11811.200195
"""H153""","""H124""",10475.544922
"""H153""","""H178""",1940168.5
"""H153""","""H142""",197.251358
"""H153""","""H179""",1818773.5
…,…,…
"""H184""","""H182""",2.0697e6
"""H184""","""H17""",1.8727e6
"""H184""","""H197""",49060.382812


In [15]:
dfa = pl.DataFrame({
    "unique_id": [1, 1, 1, 2, 2, 2],
    "y": [1, 2, 3, 4, 5, 6]
})

In [16]:
pts.compute_pairwise_dtw(dfa, dfa)

id_1,id_2,dtw
str,str,f32
"""2""","""1""",9.0
"""2""","""2""",0.0
"""1""","""1""",0.0
"""1""","""2""",9.0


In [17]:
dfa

unique_id,y
i64,i64
1,1
1,2
1,3
2,4
2,5
2,6


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def compute_dtw_matrix(a, b):
    """
    Compute the full DTW cost matrix for two sequences.
    
    Parameters:
      a (list or np.array): First time series.
      b (list or np.array): Second time series.
      
    Returns:
      dp (np.array): A (n+1) x (m+1) DTW cost matrix.
    """
    n = len(a)
    m = len(b)
    dp = np.full((n + 1, m + 1), np.inf)
    dp[0, 0] = 0

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            cost = abs(a[i - 1] - b[j - 1])
            dp[i, j] = cost + min(dp[i - 1, j],    # Insertion
                                  dp[i, j - 1],    # Deletion
                                  dp[i - 1, j - 1])  # Match
    return dp

def backtrack_dtw(dp):
    """
    Backtrack the DTW cost matrix to extract the optimal warping path.
    
    Parameters:
      dp (np.array): The DTW cost matrix.
      
    Returns:
      path (list of tuples): List of (i, j) indices along the optimal path.
    """
    i, j = dp.shape[0] - 1, dp.shape[1] - 1
    path = [(i, j)]
    
    while i > 0 or j > 0:
        if i == 0:
            j -= 1
        elif j == 0:
            i -= 1
        else:
            # Choose the step with the smallest cost
            choices = [dp[i - 1, j - 1], dp[i - 1, j], dp[i, j - 1]]
            argmin = np.argmin(choices)
            if argmin == 0:
                i, j = i - 1, j - 1
            elif argmin == 1:
                i -= 1
            else:
                j -= 1
        path.append((i, j))
    path.reverse()
    return path

# Define the two series.
series1 = df.filter(pl.col("unique_id") == "H1")["y"]
series2 = df.filter(pl.col("unique_id") == "H2")["y"]

# Compute the DTW cost matrix and the DTW distance.
dp = compute_dtw_matrix(series1, series2)
dtw_distance = dp[len(series1), len(series2)]
print("DTW distance:", dtw_distance)  # Should print 9

# Get the optimal warping path.
path = backtrack_dtw(dp)
# Remove the initial zero-index boundary if necessary (only keep pairs corresponding to actual data)
path = [(i, j) for (i, j) in path if i > 0 and j > 0]

# --- Plotting both series and the DTW relationships ---

# Create a figure with two subplots: one for each series.
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(8, 6))
x1 = np.arange(len(series1))
x2 = np.arange(len(series2))

# Plot each series with markers.
ax1.plot(x1, series1, marker='o', color='blue', label='Series 1')
ax1.set_title("Series 1")
ax2.plot(x2, series2, marker='o', color='red', label='Series 2')
ax2.set_title("Series 2")
ax2.set_xlabel("Index")

# For each aligned pair in the warping path, draw a connecting line.
# (We convert the data coordinates in each subplot to figure coordinates and draw lines in the figure.)
for (i, j) in path:
    # Convert DTW matrix indices (starting at 1) to series indices (starting at 0)
    x_coord1, y_coord1 = x1[i - 1], series1[i - 1]
    x_coord2, y_coord2 = x2[j - 1], series2[j - 1]
    
    # Transform data coordinates to display coordinates for each subplot.
    coord1 = ax1.transData.transform((x_coord1, y_coord1))
    coord2 = ax2.transData.transform((x_coord2, y_coord2))
    
    # Convert display coordinates to figure coordinates.
    fig_coord1 = fig.transFigure.inverted().transform(coord1)
    fig_coord2 = fig.transFigure.inverted().transform(coord2)
    
    # Create a line between the two points.
    line = plt.Line2D([fig_coord1[0], fig_coord2[0]],
                      [fig_coord1[1], fig_coord2[1]],
                      transform=fig.transFigure,
                      color='gray', linestyle='--', linewidth=1)
    fig.lines.append(line)

plt.tight_layout()
plt.show()
